In [1]:
#Exploring log huber loss and regularization

using Flux
using LinearAlgebra
using Random

Random.seed!(0)

function regression_fit(X, Y, l, r, lambda; numiters = 500)
    #     Inputs:
    #         X: input data
    #         Y: output data
    #         l: l(yhat, y)
    #         r: r(theta)
    #         lambda: regularization hyper-parameter
    #         numiters (optional): number of iterations
    data = zip(eachrow(X), eachrow(Y))
    n,d = size(X)
    theta = zeros(d)
    predicty(x) = theta'*x
    loss(x, y) = l(predicty(x), y[1]) + lambda*r(theta)
    cost(x,y) = loss(x,y) + lambda*r(theta)
    risk() = sum((cost(d...) for d in data))/n
    opt = Flux.ADAGrad()
    losses = []
    tracker() = push!(losses, risk())
    Flux.@epochs numiters Flux.train!(loss, Flux.params(theta), data, opt, cb = Flux.throttle(tracker,10))
    return theta
end

regression_fit (generic function with 1 method)

In [2]:
include("readclassjson.jl")
data = readclassjson("non_quadratic.json")

U_train = data["U_train"]
U_test = data["U_test"]
v_train = data["v_train"]
v_test = data["v_test"]

500-element Vector{Float64}:
  1.2824456524013885
  0.40519186680248087
  0.9051322711973372
 -0.7250191445764285
 -1.7089412749543174
 -0.2706352833337011
 -0.5659315282773868
  0.5368980119937256
 -0.8322031233805599
  1.3032238646012249
  0.2833982851311697
 -1.3388813501438168
 -0.06025053778168557
  ⋮
 -1.4387126928367873
 -0.06510434035183442
  1.4784808273858874
  0.563104504377098
  1.489394222895956
 -1.5808860135206684
  1.713759497595553
  4.146514488172257
  2.6660874992787136
  1.045627741937738
  0.03194983769152867
 -0.41641127336442774

In [3]:
rms(y, y_hat) = sqrt(sum((y_hat .- y).^2)/size(y, 1))
r_1(theta) = 0
lambda_1 = 1

l_qua(y_hat, y) = (y_hat .- y).^2
l_abs(y_hat, y) = abs(y_hat .- y)

function huber(r, alpha)
    if abs(r) <= alpha
        return r^2
    else
        return alpha*(2*abs(r) - alpha)
    end
end

l_hub_0(y_hat, y) = huber(y_hat - y, 0.5)
l_hub_1(y_hat, y) = huber(y_hat - y, 1)
l_hub_2(y_hat, y) = huber(y_hat - y, 2)

function log_huber(r, alpha)
    if abs(r) <= alpha
        return r^2
    else
        return alpha^2*(1 - 2*log(alpha) + log(r^2))
    end
end

l_loghub_0(y_hat, y) = log_huber(y_hat - y, 0.5)
l_loghub_1(y_hat, y) = log_huber(y_hat - y, 1)
l_loghub_2(y_hat, y) = log_huber(y_hat - y, 2)
 

theta_qua = regression_fit(U_train, v_train, l_qua, r_1, lambda_1)
train_rms_qua = rms(v_train, U_train*theta_qua)
test_rms_qua = rms(v_test, U_test*theta_qua)

theta_abs = regression_fit(U_train, v_train, l_abs, r_1, lambda_1)
train_rms_abs = rms(v_train, U_train*theta_abs)
test_rms_abs = rms(v_test, U_test*theta_abs)

theta_hub_0 = regression_fit(U_train, v_train, l_hub_0, r_1, lambda_1)
train_rms_hub_0 = rms(v_train, U_train*theta_hub_0)
test_rms_hub_0 = rms(v_test, U_test*theta_hub_0)

theta_hub_1 = regression_fit(U_train, v_train, l_hub_1, r_1, lambda_1)
train_rms_hub_1 = rms(v_train, U_train*theta_hub_1)
test_rms_hub_1 = rms(v_test, U_test*theta_hub_1)

theta_hub_2 = regression_fit(U_train, v_train, l_hub_2, r_1, lambda_1)
train_rms_hub_2 = rms(v_train, U_train*theta_hub_2)
test_rms_hub_2 = rms(v_test, U_test*theta_hub_2)

theta_loghub_0 = regression_fit(U_train, v_train, l_loghub_0, r_1, lambda_1)
train_rms_loghub_0 = rms(v_train, U_train*theta_loghub_0)
test_rms_loghub_0 = rms(v_test, U_test*theta_loghub_0)

theta_loghub_1 = regression_fit(U_train, v_train, l_loghub_1, r_1, lambda_1)
train_rms_loghub_1 = rms(v_train, U_train*theta_loghub_1)
test_rms_loghub_1 = rms(v_test, U_test*theta_loghub_1)

theta_loghub_2 = regression_fit(U_train, v_train, l_loghub_2, r_1, lambda_1)
train_rms_loghub_2 = rms(v_train, U_train*theta_loghub_2)
test_rms_loghub_2 = rms(v_test, U_test*theta_loghub_2)

┌ Info: Epoch 1
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 2
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 3
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 4
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 5
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 6
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 7
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 8
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 9
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 10
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 11
└ @ Main C:\Users\surface\.julia

┌ Info: Epoch 104
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 105
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 106
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 107
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 108
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 109
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 110
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 111
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 112
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 113
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 114
└ @ Main C:\

┌ Info: Epoch 206
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 207
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 208
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 209
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 210
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 211
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 212
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 213
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 214
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 215
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 216
└ @ Main C:\

┌ Info: Epoch 308
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 309
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 310
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 311
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 312
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 313
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 314
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 315
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 316
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 317
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 318
└ @ Main C:\

┌ Info: Epoch 410
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 411
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 412
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 413
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 414
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 415
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 416
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 417
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 418
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 419
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 420
└ @ Main C:\

...

┌ Info: Epoch 180
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 181
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 182
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 183
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 184
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 185
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 186
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 187
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 188
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 189
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 190
└ @ Main C:\

┌ Info: Epoch 282
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 283
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 284
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 285
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 286
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 287
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 288
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 289
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 290
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 291
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 292
└ @ Main C:\

┌ Info: Epoch 384
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 385
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 386
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 387
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 388
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 389
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 390
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 391
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 392
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 393
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 394
└ @ Main C:\

┌ Info: Epoch 486
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 487
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 488
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 489
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 490
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 491
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 492
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 493
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 494
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 495
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 496
└ @ Main C:\

1.6675871663808457

In [4]:
# The best based on test set is: test_rms_loghub_0.
# It represents Log Huber loss with alpha = 0.5.
# Log Huber loss is relatively insensitive to outliers in the data, 
# which might be why it performed better than the rest.

println(train_rms_qua)
println(train_rms_abs)
println(train_rms_hub_0)
println(train_rms_hub_1)
println(train_rms_hub_2)
println(train_rms_loghub_0)
println(train_rms_loghub_1)
println(train_rms_loghub_2)

println(test_rms_qua)
println(test_rms_abs)
println(test_rms_hub_0)
println(test_rms_hub_1)
println(test_rms_hub_2)
println(test_rms_loghub_0)
println(test_rms_loghub_1)
println(test_rms_loghub_2)

0.7261901466885229
0.8902819134859191
0.8225387168662255
0.7617124040798764
0.7298140296006335
1.071255663154638
0.9251164334299911
0.7539811017762739
1.6436056190636568
1.5537305277753575
1.6264970629784747
1.6371766342421086
1.651906104564322
1.5294853008977825
1.5414375661468138
1.6675871663808457


In [5]:
# The best two from the previous problem are: test_rms_loghub_0, test_rms_loghub_1.
# They represent Log Huber loss with alpha = 0.5 and 1.

lambda_2 = 10
r_qua(theta) = lambda_2 * norm(theta[2:end]).^2
r_las(theta) = lambda_2 * sum(abs.(theta[2:end]))

theta_loghub_0 = regression_fit(U_train, v_train, l_loghub_0, r_1, lambda_2)
train_rms_loghub_0 = rms(v_train, U_train*theta_loghub_0)
test_rms_loghub_0 = rms(v_test, U_test*theta_loghub_0)

theta_loghub_1 = regression_fit(U_train, v_train, l_loghub_1, r_1, lambda_2)
train_rms_loghub_1 = rms(v_train, U_train*theta_loghub_1)
test_rms_loghub_1 = rms(v_test, U_test*theta_loghub_1)

theta_loghub_0_qua = regression_fit(U_train, v_train, l_loghub_0, r_qua, lambda_2)
train_rms_loghub_0_qua = rms(v_train, U_train*theta_loghub_0_qua)
test_rms_loghub_0_qua = rms(v_test, U_test*theta_loghub_0_qua)

theta_loghub_1_qua = regression_fit(U_train, v_train, l_loghub_1, r_qua, lambda_2)
train_rms_loghub_1_qua = rms(v_train, U_train*theta_loghub_1_qua)
test_rms_loghub_1_qua = rms(v_test, U_test*theta_loghub_1_qua)

theta_loghub_0_las = regression_fit(U_train, v_train, l_loghub_0, r_las, lambda_2)
train_rms_loghub_0_las = rms(v_train, U_train*theta_loghub_0_las)
test_rms_loghub_0_las = rms(v_test, U_test*theta_loghub_0_las)

theta_loghub_1_las = regression_fit(U_train, v_train, l_loghub_1, r_las, lambda_2)
train_rms_loghub_1_las = rms(v_train, U_train*theta_loghub_1_las)
test_rms_loghub_1_las = rms(v_test, U_test*theta_loghub_1_las)


┌ Info: Epoch 1
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 2
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 3
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 4
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 5
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 6
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 7
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 8
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 9
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 10
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 11
└ @ Main C:\Users\surface\.julia

┌ Info: Epoch 104
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 105
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 106
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 107
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 108
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 109
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 110
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 111
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 112
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 113
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 114
└ @ Main C:\

┌ Info: Epoch 206
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 207
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 208
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 209
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 210
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 211
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 212
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 213
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 214
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 215
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 216
└ @ Main C:\

┌ Info: Epoch 308
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 309
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 310
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 311
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 312
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 313
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 314
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 315
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 316
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 317
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 318
└ @ Main C:\

┌ Info: Epoch 410
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 411
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 412
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 413
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 414
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 415
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 416
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 417
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 418
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 419
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 420
└ @ Main C:\

...

┌ Info: Epoch 158
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 159
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 160
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 161
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 162
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 163
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 164
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 165
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 166
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 167
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 168
└ @ Main C:\

┌ Info: Epoch 260
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 261
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 262
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 263
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 264
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 265
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 266
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 267
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 268
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 269
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 270
└ @ Main C:\

┌ Info: Epoch 362
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 363
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 364
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 365
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 366
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 367
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 368
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 369
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 370
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 371
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 372
└ @ Main C:\

┌ Info: Epoch 464
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 465
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 466
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 467
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 468
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 469
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 470
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 471
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 472
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 473
└ @ Main C:\Users\surface\.julia\packages\Flux\18YZE\src\optimise\train.jl:153
┌ Info: Epoch 474
└ @ Main C:\

1.211599358726964

In [6]:
# The best loss+regularizer combination is 
# Log Huber loss with alpha = 1 and quadratic regularization. 

println(train_rms_loghub_0)
println(train_rms_loghub_0_qua)
println(train_rms_loghub_0_las)

println(train_rms_loghub_1)
println(train_rms_loghub_1_qua)
println(train_rms_loghub_1_las)

println(test_rms_loghub_0)
println(test_rms_loghub_0_qua)
println(test_rms_loghub_0_las)

println(test_rms_loghub_1)
println(test_rms_loghub_1_qua)
println(test_rms_loghub_1_las)

1.071255663154638
1.2409769840466436
1.240983403449986
0.9251164334299911
1.2405969101425238
1.2406410488892814
1.5294853008977825
1.2111870543336938
1.2112066460006317
1.5414375661468138
1.2115783888881582
1.211599358726964
